Taking the workflow from 01_ and 02_ Andes written by Alexis Caro and adapting for just Cautín glaciers. Experiment to test climate database comparison 

* run: mamba activate oggm_env to get started

In [1]:
# python module import ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
from oggm import cfg, utils, workflow, tasks
import logging
import numpy as np
import os
import pandas

In [2]:
# oggm initialization ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
cfg.initialize(logging_level='WARNING')
cfg.PARAMS['use_multiprocessing'] = True
cfg.PARAMS['continue_on_error'] = True
cfg.PARAMS['run_mb_calibration'] = True
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['border'] = 80
cfg.PARAMS['store_fl_diagnostics'] = True

cfg.PARAMS["climate_qc_months"] = 3 # hugonnet 
cfg.PARAMS["hydro_month_nh"] = 1 # hugonnet
cfg.PARAMS["hydro_month_sh"] = 1 # hugonnet
cfg.PARAMS["max_mu_star"] = 600

cfg.PARAMS["temp_all_solid"] = 0.0
cfg.PARAMS["temp_all_liq"] = 2.0 
cfg.PARAMS["temp_melt"] = 0

cfg.PARAMS['cfl_min_dt'] = 10 # para simular glaciares con problemas

2025-06-09 14:31:58: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2025-06-09 14:31:58: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2025-06-09 14:31:58: oggm.cfg: Multiprocessing: using all available processors (N=8)
2025-06-09 14:32:02: oggm.cfg: Multiprocessing switched ON after user settings.
2025-06-09 14:32:02: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2025-06-09 14:32:02: oggm.cfg: WARNING: adding an unknown parameter `run_mb_calibration`:`True` to PARAMS.
2025-06-09 14:32:02: oggm.cfg: PARAMS['store_model_geometry'] changed from `False` to `True`.
2025-06-09 14:32:02: oggm.cfg: PARAMS['store_fl_diagnostics'] changed from `False` to `True`.
2025-06-09 14:32:02: oggm.cfg: WARNING: adding an unknown parameter `climate_qc_months`:`3` to PARAMS.
2025-06-09 14:32:02: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
2025-06-09 14:32:02: oggm.cfg: PARAMS['hydro_month_sh'] changed fr

In [3]:
# glacier call #############################
datos_rgi = pandas.read_csv('/Users/milliespencer/Desktop/CR2_OGGM_Paper/files_chile_OGGM_climate_comparison/RGI_BNA_Clusters.csv')
# call parameters
datos_param = pandas.read_csv('/Users/milliespencer/Desktop/CR2_OGGM_Paper/files_chile_OGGM_climate_comparison/LR_Pf.csv')

In [4]:
# # INICIO ITERACION por region GLACIO +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# listas de llamado
list_region = ['OT3','DA1','DA2','DA3','WA1','WA2','WA3','WA4','WA5','WA6']


for zona in list_region:
    
# directorio
#    zona = 'OT3'
    # salida = '/home/caro/04_Andes_2000_2019_CR2_2023/' + zona + '/'
    salida = '/Users/milliespencer/Desktop/CR2_OGGM_Paper/files_chile_OGGM_climate_comparison' + zona + '/'
    cfg.PATHS['working_dir'] = salida                                 

# extraccion de glaciares por zona
    gdf_sel = datos_rgi[datos_rgi['Cluster'] == zona]
    gdf_sel = list(gdf_sel.RGIId)
    
# se extrae LT usando id glaciar
    param = datos_param[datos_param['Cluster'] == zona]
    LT_n = float(param.LR)
# se extrae Pf 
    Pf_n = float(param.Pf)

    # definion LT en modelo
    cfg.PARAMS["temp_default_gradient"] = LT_n
    cfg.PARAMS["prcp_scaling_factor"] = Pf_n
    cfg.PARAMS['use_winter_prcp_factor'] = False
    

/var/folders/3j/6dy_9gxj7vvgct178jkkp1680000gn/T/ipykernel_97788/1109579582.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  LT_n = float(param.LR)
/var/folders/3j/6dy_9gxj7vvgct178jkkp1680000gn/T/ipykernel_97788/1109579582.py:23: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Pf_n = float(param.Pf)
2025-06-09 14:32:07: oggm.cfg: WARNING: adding an unknown parameter `prcp_scaling_factor`:`1.0` to PARAMS.
2025-06-09 14:32:07: oggm.cfg: WARNING: adding an unknown parameter `use_winter_prcp_factor`:`False` to PARAMS.
2025-06-09 14:32:07: oggm.cfg: PARAMS['temp_default_gradient'] changed from `-0.0065` to `-0.008`.
2025-06-09 14:32:07: oggm.cfg: PARAMS['prcp_scaling_factor'] changed from `1.0` to `2.0`.
2025-06-09 14:32:07: oggm.cfg: PARAMS['temp_default_gradient'] changed from `-0.008` to `-

In [5]:
###
### LLAMADO PERIODO 2000-2019 +++++++++++++++++++++++++++++++++++++++++++++++++
###
    # Guardado statistics
output_folder= salida
rgi_version  = '_'+ zona
border = 80
output_base_dir = os.path.join(output_folder,'RGI{}'.format(rgi_version), 'b_{:03d}'.format(border))
sum_dir = os.path.join(output_base_dir, 'L3', 'summary')
utils.mkdir(sum_dir)

'/Users/milliespencer/Desktop/CR2_OGGM_Paper/files_chile_OGGM_climate_comparisonWA6/RGI_WA6/b_080/L3/summary'

In [ ]:
    # Level = 2 2000-2019+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#    gdirs = workflow.init_glacier_directories(gdf_sel, from_prepro_level=2)
    
gdirs = workflow.init_glacier_directories(gdf_sel, from_prepro_level=2,
                                              prepro_base_url='https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/centerlines/',
                                              prepro_border=80)

2025-06-09 14:48:44: oggm.workflow: init_glacier_directories from prepro level 2 on 2671 glaciers.
2025-06-09 14:48:44: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 2671 glaciers


In [ ]:
# # INICIO ITERACION por region GLACIO +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# listas de llamado
list_region = ['OT3','DA1','DA2','DA3','WA1','WA2','WA3','WA4','WA5','WA6']


for zona in list_region:
    
# directorio
#    zona = 'OT3'
    # salida = '/home/caro/04_Andes_2000_2019_CR2_2023/' + zona + '/'
    salida = '/Users/milliespencer/Desktop/files_chile_OGGM_climate_comparison' + zona + '/'
    cfg.PATHS['working_dir'] = salida                                 

# extraccion de glaciares por zona
    gdf_sel = datos_rgi[datos_rgi['Cluster'] == zona]
    gdf_sel = list(gdf_sel.RGIId)
    
# se extrae LT usando id glaciar
    param = datos_param[datos_param['Cluster'] == zona]
    LT_n = float(param.LR)
# se extrae Pf 
    Pf_n = float(param.Pf)

    # definion LT en modelo
    cfg.PARAMS["temp_default_gradient"] = LT_n
    cfg.PARAMS["prcp_scaling_factor"] = Pf_n
    cfg.PARAMS['use_winter_prcp_factor'] = False
    
        
###
### LLAMADO PERIODO 2000-2019 +++++++++++++++++++++++++++++++++++++++++++++++++
###
    # Guardado statistics
    output_folder= salida
    rgi_version  = '_'+ zona
    border = 80
    output_base_dir = os.path.join(output_folder,'RGI{}'.format(rgi_version), 'b_{:03d}'.format(border))
    sum_dir = os.path.join(output_base_dir, 'L3', 'summary')
    utils.mkdir(sum_dir)

    # Level = 2 2000-2019+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#    gdirs = workflow.init_glacier_directories(gdf_sel, from_prepro_level=2)
    
    gdirs = workflow.init_glacier_directories(gdf_sel, from_prepro_level=2,
                                              prepro_base_url='https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/centerlines/',
                                              prepro_border=80)
    
       # Climate CR2
    cfg.PARAMS['baseline_climate'] = 'CR2MET25'
    from oggm.shop import cr2met_25
    workflow.execute_entity_task(cr2met_25.process_cr2met_25_data, gdirs) # ,output_filesuffix='_cr2met'
    utils.get_geodetic_mb_dataframe()  # Small optim to avoid concurrency
    workflow.execute_entity_task(tasks.mu_star_calibration_from_geodetic_mb, gdirs,ref_period='2000-01-01_2020-01-01') # edit climate.py Between lines 539 and 554
    workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs)
    
    
       # Climate ERA5
    cfg.PARAMS['baseline_climate'] = 'ERA5'
    from oggm.shop import ecmwf
    workflow.execute_entity_task(ecmwf.process_ecmwf_data, gdirs) # ,output_filesuffix='_cr2met'
    utils.get_geodetic_mb_dataframe()  # Small optim to avoid concurrency
    workflow.execute_entity_task(tasks.mu_star_calibration_from_geodetic_mb, gdirs,ref_period='2000-01-01_2020-01-01') # edit climate.py Between lines 539 and 554
    workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs)

    filter = border >= 20
    workflow.calibrate_inversion_from_consensus(gdirs,apply_fs_on_mismatch=True,error_on_mismatch=False,filter_inversion_output=filter)
         
        # We get ready for modelling
    log = logging.getLogger(__name__)     
    if border >= 20:
                workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs)
    else:
        log.workflow('L3: for map border values < 20, wont initialize glaciers '
                      'for the run.')
    
    compilacion = '_2000_2019_hydro_TC_'+ zona 
    # save outputs 2000-2019
    workflow.execute_entity_task (tasks.run_with_hydro, gdirs,
                         ys=1999,     
                         run_task = tasks.run_from_climate_data,          # The task to run with hydro
                         store_monthly_hydro = True,                      # compute monthly hydro diagnostics
                         ref_area_from_y0 = True,                           # Even if the glacier may grow, keep the reference area as the year 0 of the simulation
                         output_filesuffix = compilacion,                # Where to write the output - this is needed to stitch the runs together afterwards
                         );

    # Guardado glacier statistics 
    rgi_reg = '_' + zona
    opath = os.path.join(sum_dir, 'glacier_statistics_{}.csv'.format(rgi_reg))
    utils.compile_glacier_statistics(gdirs, path=opath)
    mb_dif = utils.compile_glacier_statistics(gdirs, path=opath)
    opath = os.path.join(sum_dir, 'climate_statistics_{}.csv'.format(rgi_reg))
    utils.compile_climate_statistics(gdirs, path=opath)
    opath = os.path.join(sum_dir, 'fixed_geometry_mass_balance_{}.csv'.format(rgi_reg))
    utils.compile_fixed_geometry_mass_balance(gdirs, path=opath)
    
    # compilación datos en un archivo
    ds2000 = utils.compile_run_output(gdirs, input_filesuffix= compilacion )

/var/folders/3j/6dy_9gxj7vvgct178jkkp1680000gn/T/ipykernel_97355/3174518385.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  LT_n = float(param.LR)
/var/folders/3j/6dy_9gxj7vvgct178jkkp1680000gn/T/ipykernel_97355/3174518385.py:23: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Pf_n = float(param.Pf)
2025-05-22 17:39:37: oggm.workflow: init_glacier_directories from prepro level 2 on 0 glaciers.
2025-05-22 17:39:37: oggm.workflow: Called execute_entity_task on 0 glaciers. Returning...
2025-05-22 17:39:37: oggm.cfg: PARAMS['baseline_climate'] changed from `ERA5L` to `W5E5`.
2025-05-22 17:39:37: oggm.workflow: Execute entity tasks [process_ecmwf_data] on 1 glaciers


AttributeError: 'NoneType' object has no attribute 'get_task_status'